# Asthma

## Imports

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
from glob import glob
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd 
import seaborn as sns
import matplotlib
from math import pow, sqrt, ceil
from collections import OrderedDict

from sklearn.metrics import accuracy_score, cohen_kappa_score, confusion_matrix

In [3]:
from Analysis.annotations import Annotation
from Analysis.image import Image
from Analysis.expert import Expert, DatasetType, ProjectType

In [4]:
from utils import *

## Load Annotations

In [5]:
path = Path("Results/Asthma/")
files = glob(str(path)+"/*/Participants/*.txt")
gt_path = str(path)+"/FirstRound/GT/ground truth.txt"

users_first_round = {}
users_second_round = {}

ground_truth = Expert("GT", BBType.GroundTruth, DatasetType.EIPH_Exact, ProjectType.GroundTruth)
ground_truth.add_file(gt_path)

deleted_path = str(path)+"/FirstRound/GT/deleted.txt"
changed_path = str(path)+"/FirstRound/GT/changed.txt"
nms_path = str(path)+"/FirstRound/GT/nms.txt"


deleted_eiph = Expert("deleted", BBType.GroundTruth, DatasetType.EIPH_Exact, ProjectType.GroundTruth)
deleted_eiph.add_file(deleted_path)

changed_eiph = Expert("changed", BBType.GroundTruth, DatasetType.EIPH_Exact, ProjectType.GroundTruth)
changed_eiph.add_file(changed_path)

nms_eiph = Expert("changed", BBType.GroundTruth, DatasetType.EIPH_Exact, ProjectType.GroundTruth)
nms_eiph.add_file(nms_path)

for file_name in files:
    participant = "Expert_" + Path(file_name).stem.split("_")[-1]
    
    if "FirstRound" in file_name:
        expert = Expert(participant, BBType.Detected, DatasetType.EIPH_Exact, ProjectType.Annotation)
        expert.add_file(file_name)
        
        users_first_round[expert.expert] = expert
    else:
        expert = Expert(participant, BBType.Detected, DatasetType.EIPH_Exact, ProjectType.ExpertAlgorithm)
        expert.add_file(file_name)
        
        users_second_round[expert.expert] = expert

## Statistics

In [6]:
print("First Round")
for user in users_first_round.values():
    print(str(user) + " mIoU: {}".format(user.calc_MIoU(ground_truth)))
    
print("\nSecond Round")    
for user in users_second_round.values():
    print(str(user) + " mIoU: {}".format(user.calc_MIoU(ground_truth)))

First Round
Expert_8:  Annos: 408 Seconds: 114.5 Type: EIPH_Exact Project: Annotation mIoU: 0.765555282730989
Expert_9:  Annos: 410 Seconds: 164.76190476190476 Type: EIPH_Exact Project: Annotation mIoU: 0.7422009626832827
Expert_10:  Annos: 402 Seconds: 113.38095238095238 Type: EIPH_Exact Project: Annotation mIoU: 0.7753388947263968
Expert_1:  Annos: 397 Seconds: 56.2 Type: EIPH_Exact Project: Annotation mIoU: 0.5641239801706175
Expert_2:  Annos: 413 Seconds: 140.25 Type: EIPH_Exact Project: Annotation mIoU: 0.7774293463099948
Expert_3:  Annos: 406 Seconds: 96.6 Type: EIPH_Exact Project: Annotation mIoU: 0.8226910144113211
Expert_4:  Annos: 402 Seconds: 72.8 Type: EIPH_Exact Project: Annotation mIoU: 0.7835845035027322
Expert_5:  Annos: 409 Seconds: 95.8 Type: EIPH_Exact Project: Annotation mIoU: 0.5932228978718947
Expert_6:  Annos: 410 Seconds: 85.15 Type: EIPH_Exact Project: Annotation mIoU: 0.7490406825045358
Expert_7:  Annos: 406 Seconds: 126.5 Type: EIPH_Exact Project: Annotation 

In [7]:
expert_names = set([expert for expert in users_first_round] + [expert for expert in users_second_round])
expert_names

{'Expert_1',
 'Expert_10',
 'Expert_2',
 'Expert_3',
 'Expert_4',
 'Expert_5',
 'Expert_6',
 'Expert_7',
 'Expert_8',
 'Expert_9'}

In [8]:
metrics_changed = []
metrics_deleted = []
metrics_nms = []

for expert_name in expert_names:
    
    first_round_expert = users_first_round[expert_name]
    second_round_expert = users_second_round[expert_name] if expert_name in users_second_round else None
    
    total_changed_cells = 0
    total_deleted_cells = 0
    total_nms_cells = 0
    
    first_round_changed_cells = 0
    first_round_deleted_cells = 0
    first_round_nms_cells = 0
    
    second_round_changed_cells = 0
    second_round_deleted_cells = 0
    second_round_nms_cells = 0
    
    if first_round_expert:
        for metric in first_round_expert.calc_metrics(changed_eiph):
            total_changed_cells += int(metric['total positives'])
            first_round_changed_cells += int(metric['total TP'])
            
        for metric in first_round_expert.calc_metrics(deleted_eiph):
            total_deleted_cells += int(metric['total positives'])
            first_round_deleted_cells += int(metric['total TP'])
    
        for metric in user.calc_metrics(nms_eiph):
            total_nms_cells += int(metric['total positives'])
            first_round_nms_cells += int(metric['total TP'])
            
    if second_round_expert:
        for metric in second_round_expert.calc_metrics(changed_eiph):
            second_round_changed_cells += int(metric['total TP'])
            
        for metric in second_round_expert.calc_metrics(deleted_eiph):
            second_round_deleted_cells += int(metric['total TP'])
            
        for metric in second_round_expert.calc_metrics(nms_eiph):
            second_round_nms_cells += int(metric['total TP'])
            
    metrics_changed.append([expert_name, total_changed_cells, first_round_changed_cells, second_round_changed_cells])
    metrics_deleted.append([expert_name, total_deleted_cells, first_round_deleted_cells, second_round_deleted_cells])
    metrics_nms.append([expert_name, total_nms_cells, first_round_nms_cells, second_round_nms_cells])

#metrics_changed.append(["Total"] + [sum([row[i] for row in metrics_changed]) for i in range(1, 4)])
#metrics_deleted.append(["Total"] + [sum([row[i] for row in metrics_deleted]) for i in range(1, 4)])
#metrics_nms.append(["Total"] + [sum([row[i] for row in metrics_nms]) for i in range(1, 4)])


metrics_changed = pd.DataFrame(metrics_changed, columns=['Name','Total Changed', "Annotation", "ExpertAlgorithm"])

metrics_deleted = pd.DataFrame(metrics_deleted, columns=['Name',"Total Deleted", "Annotation", "ExpertAlgorithm"])

metrics_nms = pd.DataFrame(metrics_nms, columns=['Name', "Total NMS", "Annotation", "ExpertAlgorithm"])

D:\ProgProjekte\Python\Object-Detection-Metrics\lib\Evaluator.py:126: RuntimeWarning: invalid value encountered in true_divide
  rec = acc_TP / npos


### Changed Cells

In [9]:
metrics_changed

,Name,Total Changed,Annotation,ExpertAlgorithm
0,Expert_9,15,13,12
1,Expert_5,15,11,3
2,Expert_10,15,15,9
3,Expert_3,15,12,14
4,Expert_6,15,12,15
5,Expert_2,15,14,15
6,Expert_1,15,13,13
7,Expert_4,15,13,12
8,Expert_7,15,13,13
9,Expert_8,15,13,12


In [10]:
print("Annotation: {0}%".format(((np.sum(metrics_changed['Annotation']) / (np.mean(metrics_changed['Total Changed']) * len(users_first_round))) * 100)))

Annotation: 86.0%


In [11]:
print("ExpertAlgorithm: {0}%".format(((np.sum(metrics_changed['ExpertAlgorithm']) / (np.mean(metrics_changed['Total Changed']) * len(users_second_round))) * 100)))

ExpertAlgorithm: 78.66666666666666%


### Deleted Cells

In [12]:
metrics_deleted

,Name,Total Deleted,Annotation,ExpertAlgorithm
0,Expert_9,5,4,4
1,Expert_5,5,4,1
2,Expert_10,5,3,3
3,Expert_3,5,5,4
4,Expert_6,5,3,4
5,Expert_2,5,3,5
6,Expert_1,5,5,5
7,Expert_4,5,5,4
8,Expert_7,5,5,5
9,Expert_8,5,5,4


In [13]:
print("Annotation: {0}%".format(((np.sum(metrics_deleted['Annotation']) / (np.mean(metrics_deleted['Total Deleted']) * len(users_first_round))) * 100)))

Annotation: 84.0%


In [14]:
print("ExpertAlgorithm: {0}%".format(((np.sum(metrics_deleted['ExpertAlgorithm']) / (np.mean(metrics_deleted['Total Deleted']) * len(users_second_round))) * 100)))

ExpertAlgorithm: 78.0%


### NMS Cells

In [15]:
metrics_nms

,Name,Total NMS,Annotation,ExpertAlgorithm
0,Expert_9,8,5,6
1,Expert_5,8,5,8
2,Expert_10,8,5,6
3,Expert_3,8,5,4
4,Expert_6,8,5,6
5,Expert_2,8,5,4
6,Expert_1,8,5,7
7,Expert_4,8,5,4
8,Expert_7,8,5,5
9,Expert_8,8,5,4


In [16]:
print("Annotation: {0}%".format(((np.sum(metrics_nms['Annotation']) / (np.mean(metrics_nms['Total NMS']) * len(users_first_round))) * 100)))

Annotation: 62.5%


In [17]:
print("ExpertAlgorithm: {0}%".format(((np.sum(metrics_nms['ExpertAlgorithm']) / (np.mean(metrics_nms['Total NMS']) * len(users_second_round))) * 100)))

ExpertAlgorithm: 67.5%
